In [1]:
%matplotlib inline 
import numpy as np
import matplotlib.pyplot as plt

# 1. Cargamos el conjunto de entrenamiento
Cargamos nuestro Conjunto de entrenamiento:

Cargamos los positivos

In [2]:
from skimage import io
from skimage.color import rgb2gray

positive_patches = list()

path = "..\\imgs\\"
for i in range(376):
    for j in range(63):
        image = io.imread(path+str(i)+str(j)+".jpg")
        image = rgb2gray(image)
        positive_patches.append(image)

Cargamos los negativos.

In [3]:
from skimage import feature, color, data, transform

imgs_to_use = ['camera', 'text', 'coins', 'moon',
               'page', 'clock', 'immunohistochemistry',
               'chelsea', 'coffee', 'hubble_deep_field']
images = [color.rgb2gray(getattr(data, name)())
          for name in imgs_to_use]

import numpy as np
from sklearn.feature_extraction.image import PatchExtractor

def extract_patches(img, N, scale=1.0, patch_size=positive_patches[0].shape):
    extracted_patch_size = tuple((scale * np.array(patch_size)).astype(int))
    extractor = PatchExtractor(patch_size=extracted_patch_size,
                               max_patches=N, random_state=0)
    patches = extractor.transform(img[np.newaxis])
    if scale != 1:
        patches = np.array([transform.resize(patch, patch_size)
                            for patch in patches])
    return patches

In [4]:
negative_patches = np.vstack([extract_patches(im, 1000, scale, patch_size=(62,47))
                              for im in images for scale in [0.5, 1.0, 2.0]])

from itertools import chain
positive_patches = np.array(positive_patches)
X_train = np.vstack((positive_patches,negative_patches))
y_train = np.zeros(X_train.shape[0])
y_train[:positive_patches.shape[0]] = 1

In [5]:
print(positive_patches.shape,negative_patches.shape)

(23688, 62, 47) (30000, 62, 47)


## Serializamos X e Y

In [6]:
import pickle # Módulo para serializar

path = '..//..//rsc//obj//'

real_x_path = path + 'real_x_train.sav'
real_y_path = path + 'real_y_train.sav'

pickle.dump(X_train, open(real_x_path, 'wb'))
pickle.dump(y_train, open(real_y_path, 'wb'))